In [87]:
grid = []

with open("input.txt") as f:
    for line in f:
        grid.append(line.strip())

len(grid), len(grid[1]), grid[1][1]

(140, 140, '7')

In [88]:
start_loc = None

for x, line in enumerate(grid):
    y = line.find("S")
    if y != -1:
        start_loc = (x, y)
        break

start_loc

(49, 96)

In [89]:
UP = (-1, 0)
RIGHT = (0, 1)
DOWN = (1, 0)
LEFT = (0, -1)

POSSIBLE_TO_CONNECTIONS = {
    UP: ["|", "F", "7", "S"],
    DOWN: ["|", "L", "J", "S"],
    LEFT: ["-", "F", "L", "S"],
    RIGHT: ["-", "7", "J", "S"],
}


def can_connect(from_pipe, to_pipe, direction):
    if from_pipe == "S":
        return to_pipe in POSSIBLE_TO_CONNECTIONS[direction]
    elif from_pipe == "|" and direction in [UP, DOWN]:
        return to_pipe in POSSIBLE_TO_CONNECTIONS[direction]
    elif from_pipe == "-" and direction in [LEFT, RIGHT]:
        return to_pipe in POSSIBLE_TO_CONNECTIONS[direction]
    elif from_pipe == "L" and direction in [UP, RIGHT]:
        return to_pipe in POSSIBLE_TO_CONNECTIONS[direction]
    elif from_pipe == "J" and direction in [UP, LEFT]:
        return to_pipe in POSSIBLE_TO_CONNECTIONS[direction]
    elif from_pipe == "7" and direction in [DOWN, LEFT]:
        return to_pipe in POSSIBLE_TO_CONNECTIONS[direction]
    elif from_pipe == "F" and direction in [DOWN, RIGHT]:
        return to_pipe in POSSIBLE_TO_CONNECTIONS[direction]
    else:
        return False


def test_can_connect():
    assert can_connect("S", "F", UP)
    assert can_connect("S", "|", DOWN)
    assert can_connect("S", "F", LEFT)
    assert can_connect("S", "-", RIGHT)
    assert can_connect("|", "F", UP)
    assert can_connect("|", "7", UP)
    assert can_connect("|", "|", UP)
    assert not can_connect("|", "-", UP)
    assert not can_connect("|", "L", UP)
    assert not can_connect("|", "J", UP)

In [90]:
def find_next_connected_pipe(grid, prev, current):
    # print(f"Looking for next connected pipe from {current} with prev {prev}")
    x, y = current
    for dx, dy in [UP, RIGHT, DOWN, LEFT]:
        nx, ny = x + dx, y + dy
        if nx < 0 or nx >= len(grid) or ny < 0 or ny >= len(grid[nx]):
            continue
        if (nx, ny) == prev:
            continue
        if can_connect(grid[x][y], grid[nx][ny], (dx, dy)):
            return (nx, ny), (dx, dy)


# print(f"UP: {UP}, RIGHT: {RIGHT}, DOWN: {DOWN}, LEFT: {LEFT}")


prev = None
current = start_loc
next = (0, 0)
loop = []
while grid[current[0]][current[1]] != "S" or len(loop) == 0:
    next, direction = find_next_connected_pipe(grid, prev, current)
    loop.append(next)
    prev = current
    current = next

len(loop) // 2

7066

# Part 2

In [105]:
# Saw on reddit shoelace + picks theorem 🤯
sum = 0
for i in range(len(loop)):
    n_1 = loop[i]
    n_2 = loop[(i + 1) % len(loop)]
    x_1, y_1 = n_1
    x_2, y_2 = n_2
    sum += x_1 * y_2 - y_1 * x_2

area = abs(sum / 2)

int(area - len(loop) / 2 + 1)

401

## My futile attempts at a solution

In [ ]:
inner = []
is_prev_loop_part = False
are_we_in_enclosed = False

for x in range(len(grid)):
    for y in range(len(grid[x])):
        if are_we_in_enclosed:
            inner.append((x, y))
            is_prev_loop_part = False
        if (x, y) in loop:
            if not is_prev_loop_part:
                are_we_in_enclosed = not are_we_in_enclosed
            is_prev_loop_part = True

len(inner)

In [92]:
# Wrong thinking:

innerParts = []

for pipe in loop:
    # print(f"Checking pipe {pipe}")
    x, y = pipe
    next = (x, y + 1)
    nx, ny = next
    newInnerParts = []
    while True:
        if next not in loop:
            if ny >= len(grid[nx]):
                break
            # print(f"Found inner part at {next}")
            newInnerParts.append(next)
            next = (nx, ny + 1)
            nx, ny = next
        else:
            # print(f"Adding inner parts")
            innerParts.extend(newInnerParts)
            break

len(innerParts)

1346

In [93]:
with open("output.txt", "w") as f:
    for x in range(len(grid)):
        for y in range(len(grid[x])):
            if (x, y) in loop:
                f.write(grid[x][y])
            elif (x, y) in innerParts:
                f.write("I")
            else:
                f.write(" ")
        f.write("\n")

In [94]:
from PIL import Image

# Create a new image with RGB mode
width = len(grid)
height = len(grid[0])
img = Image.new("RGB", (width, height))

# Access the pixel data
pixels = img.load()

# Loop over each pixel
for i in range(width):
    for j in range(height):
        if (i, j) in loop:  # Replace with your condition
            pixels[i, j] = (0, 0, 0)  # Black
        elif (i, j) in inner:  # Replace with your condition
            pixels[i, j] = (0, 255, 0)  # Green
        else:
            pixels[i, j] = (255, 255, 255)  # White

# Save the image
img.save("output2.png")